## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics, linear_model
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [2]:
# load data
digits = datasets.load_digits()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=4)

# 建立模型
clf_GBC = GradientBoostingClassifier()

# 訓練模型
clf_GBC.fit(x_train, y_train)

# 預測測試集
y_pred_GBC = clf_GBC.predict(x_test)

In [3]:
acc_GBC = metrics.accuracy_score(y_test, y_pred_GBC)
print("Acuuracy(acc_GBC): ", acc_GBC)

Acuuracy(acc_GBC):  0.9666666666666667


In [4]:
# 設定要訓練的超參數組合
n_estimators = [100,150, 200,250,300,350,400]
max_depth = [1,2, 3,4,5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf_GBC, param_grid, scoring=metrics.make_scorer(metrics.accuracy_score), n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 35 種參數組合，總共要 train 105 次模型

Fitting 3 folds for each of 35 candidates, totalling 105 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:  1.1min finished


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.965108 using {'max_depth': 2, 'n_estimators': 300}


In [6]:
grid_result.best_params_

{'max_depth': 2, 'n_estimators': 300}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [8]:
# 調整參數後可提升至 0.98 的 score
print(metrics.accuracy_score(y_test, y_pred))

0.98
